### Morans I

In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import pandas as pd
import geopandas as gpd

from utils import specific_polygon, plot_hex_grid, calculate_gi, plot_gi, read_data, read_taiwan_specific

combined_data = read_data()
taiwan, grid_filter = read_taiwan_specific(read_grid=True)

In [ ]:
area = {
    '臺北市': None,
    '新北市': None,
    '臺中市': None,
    '高雄市': None,
    '花蓮縣': None,
    '臺東縣': None
}

for area_name in area.keys():
    print(f'Processing {area_name}...')
    specific, taiwan_specific = specific_polygon(combined_data, taiwan, [area_name])
    taiwan_specific.to_crs(epsg=3826, inplace=True)
    hex_grid = plot_hex_grid(specific, taiwan_specific, threshold=-1, hex_size=0.001)
    area[area_name] = {
        'hex_grid': hex_grid,
        'specific_data': taiwan_specific
    }


In [ ]:
import pickle
with open("../ComputedDataV2/Grid/area_for_micro_analysis.pkl", "wb") as f:
    pickle.dump(area, f)

In [40]:
county = '臺東縣'
hex_grid = area[county]['hex_grid']
taiwan_specific = area[county]['specific_data']

## Lisa plot

In [ ]:
from utils_macro import LocalMoranAnalysis

lma = LocalMoranAnalysis(hex_grid, taiwan_specific, k=6)
lma.calculate_local_moran()
lma.plot_lisa()
lma.lisa_scatter_plot()
# lma.plot_lisa_folium()

### Gestis-Ord

In [41]:
grid = calculate_gi(6, hex_grid, adjacency='knn')
# plot_gi(taiwan_specific.to_crs(epsg=4326), grid.to_crs(epsg=4326))
# grid.to_file('../ComputedData/Grid/micro_north_gi.geojson', driver='GeoJSON')

### Add Infrastructure on the map

In [10]:
MRT_full = pd.read_csv('../ComputedData/MRT/full_mrt.csv')
Youbike_full = pd.read_csv('../ComputedData/Youbike/full_youbike.csv')
Parkinglot_full = pd.read_csv('../ComputedData/Parkinglot/full_parkinglot.csv')

def group_infrastructure_count(df, hex_grid):
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df['PositionLon'], df['PositionLat']),
        crs='EPSG:4326'
    )
    gdf = gdf.to_crs(hex_grid.crs)
    joined = gpd.sjoin(gdf, hex_grid, how='left', predicate='within')
    mrt_count = joined.groupby('index_right').size()
    
    return mrt_count

In [42]:
mrt_count = group_infrastructure_count(MRT_full, hex_grid)
youbike_count = group_infrastructure_count(Youbike_full, hex_grid)
parkinglot_count = group_infrastructure_count(Parkinglot_full, hex_grid)

hex_grid['mrt_count'] = hex_grid.index.map(mrt_count).fillna(0).astype(int)
hex_grid['youbike_count'] = hex_grid.index.map(youbike_count).fillna(0).astype(int)
hex_grid['parkinglot_count'] = hex_grid.index.map(parkinglot_count).fillna(0).astype(int)
hex_grid['full_infrastructure'] = hex_grid['mrt_count'] + hex_grid['youbike_count'] + hex_grid['parkinglot_count']

In [ ]:
from utils import plot_map
plot_map(specific, hex_grid.to_crs(epsg=4326), gi=True, count='full_infrastructure')

### 觀察設施的多寡是否和事故的數量有相關

In [ ]:
from utils_micro import scatter_with_regression, scatter_with_spearman

scatter_with_regression(hex_grid, 'num_accidents', 'full_infrastructure')
scatter_with_spearman(hex_grid, 'num_accidents', 'full_infrastructure')

### 觀察全部設施是否和人與車有相關

In [ ]:
combined_gdf = gpd.GeoDataFrame(
    combined_data,
    geometry=gpd.points_from_xy(combined_data['經度'], combined_data['緯度']),
    crs='EPSG:4326'
)
combined_gdf = combined_gdf.to_crs(hex_grid.crs)
joined = gpd.sjoin(combined_gdf, hex_grid, how='left', predicate='within')

feature = '事故類型及型態大類別名稱'
target = '人與車'

### Correlation

In [ ]:
from utils_macro import calculate, calculate_most_common, calculate_average

data_feature = joined.groupby('index_right').apply(lambda group: calculate(group, feature, target, ratio=True))
hex_grid['human_vehicle_total'] = hex_grid.index.map(data_feature).fillna(0)

scatter_with_regression(hex_grid, 'human_vehicle_total', 'full_infrastructure')
scatter_with_spearman(hex_grid, 'human_vehicle_total', 'full_infrastructure')

### Cross classification plot

In [43]:
hex_grid['new_hotspot'] = hex_grid['hotspot'].apply(lambda x: 'Hotspot' if x != 'Not Significant' else 'Not Significant')
hex_grid['new_full_infrastructure'] = hex_grid['full_infrastructure'].apply(lambda x: 'Include Infrastructure' if x > 0 else 'No Infrastructure')
hex_grid['type'] = hex_grid['new_hotspot'] + '/' + hex_grid['new_full_infrastructure']

In [44]:
from utils_micro import plot_map_type

map_result = plot_map_type(specific, hex_grid.to_crs(epsg=4326))
map_result.save('map.html')

1. Pedestrian-car accident ratio in each County/City
2. Bicylce accident ratio in each County/City

In [ ]:
combined_gdf = gpd.GeoDataFrame(
    combined_data,
    geometry=gpd.points_from_xy(combined_data['經度'], combined_data['緯度']),
    crs='EPSG:4326'
)

taiwan_crs = taiwan.to_crs('EPSG:4326')
combined_gdf_crs = combined_gdf.to_crs('EPSG:4326')
joined = gpd.sjoin(combined_gdf_crs, taiwan_crs, how='left', predicate='within')

In [ ]:
combined_data['縣市'] = joined['COUNTYNAME']
city_human_vehicle_ratio = {}
cities = ['臺北市', '新北市', '臺中市', '高雄市', '花蓮縣', '臺東縣']

for city in cities:
    city_data = combined_data[combined_data['縣市'] == city]
    if len(city_data) > 0:
        human_vehicle_count = len(city_data[city_data['事故類型及型態大類別名稱'] == '人與車'])
        # human_vehicle_count = len(city_data[city_data['當事者區分-類別-子類別名稱-車種'] == '腳踏自行車'])
        total_count = len(city_data)
        ratio = human_vehicle_count / total_count
        city_human_vehicle_ratio[city] = ratio
        print(f"{city}: {total_count} 件事故, 人車事故比例: {ratio:.3f}")
    else:
        city_human_vehicle_ratio[city] = 0
        print(f"{city}: 0 件事故")

In [ ]:
import matplotlib.pyplot as plt
from config import countycity_dct

ratio_df = pd.DataFrame(list(city_human_vehicle_ratio.items()), columns=['City', 'Human_Vehicle_Ratio'])
ratio_df = ratio_df[ratio_df['Human_Vehicle_Ratio'] > 0]
ratio_df = ratio_df.sort_values('Human_Vehicle_Ratio', ascending=True)
ratio_df['City'] = ratio_df['City'].map(countycity_dct)

plt.figure(figsize=(14, 8))
bars = plt.bar(ratio_df['City'], ratio_df['Human_Vehicle_Ratio'], 
               color='steelblue', alpha=0.7)

average_ratio = ratio_df['Human_Vehicle_Ratio'].mean()
# plt.axhline(y=average_ratio, color="#6060cf", linestyle='--', linewidth=2)
# plt.text(len(ratio_df) - 1, average_ratio + 0.003, f'Avg: {average_ratio:.3f}', 
#          color='#6060cf', fontweight='bold', fontsize=12)

plt.rcParams['font.family'] = ['Microsoft JhengHei'] 
# plt.title('Bike accident ratio in each County/City', fontsize=16, fontweight='bold') # Pedestrian-car
plt.xlabel('County/City', fontsize=12)
plt.ylabel('Pedestrian-car accident ratio', fontsize=12)
plt.xticks(rotation=0, ha='center')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()